In [1]:
import miner2.miner2
import miner2.miner2.preprocess
import miner2.miner2.coexpression
import miner2.miner2.mechanistic_inference
import pandas as pd
import os

2019-08-07 10:48:05 	 hello from development


In [2]:
df = pd.read_csv('/home/aliu/omics4tb2/aliu/projects/causalAssociation/results/expected/GSM3587977_AML707B-D97.dem.txt', sep='\t', header = 0, index_col = 0, engine='python')
print(df.values.max())

PermissionError: [Errno 13] Permission denied

In [5]:
# create name for results folder where output files will be saved
resultsFolder = "results/miner/GSM3587977/"

# create results directory
resultsDirectory = os.path.join(os.path.split(os.getcwd())[0],resultsFolder)
if not os.path.isdir(resultsDirectory):
    os.mkdir(resultsDirectory)
    
print(os.path.join(os.path.split(os.getcwd())[0],resultsFolder))

/home/aliu/Projects/causalAssociation/results/miner/GSM3587977/


In [3]:
EXPRESSION_DATA_FILE = '/home/aliu/omics4tb2/aliu/projects/causalAssociation/results/expected/GSM3587977_AML707B-D97.dem.txt'
expression_data, conversion_table = miner2.miner2.preprocess.main(EXPRESSION_DATA_FILE)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.